In [ ]:
!pip install torch
!pip3 install torchvision

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import os
from torch.utils.data import Dataset
import cv2
from tqdm import tqdm
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
class UnetModel(nn.Module):
  def conv(self, in_channels, out_channels):
    block = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=(1,1)),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=3,padding=(1,1)),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )
    return block
  
  def up_conv(self, in_channels, out_channels):
    block = nn.Sequential(
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels, out_channels, kernel_size=3,padding=(1,1)),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )
    return  block
  
  def __init__(self, in_channel, out_channel):
    super(UnetModel, self).__init__()
    
    self.conv1 = self.conv(in_channel,64)
    self.conv1_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = self.conv(64, 128)
    self.conv2_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv3 = self.conv(128, 256)
    self.conv3_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv4 = self.conv(256, 512)
    self.conv4_maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv5 = self.conv(512, 1024)
    
    self.up_conv4 = self.up_conv(1024, 512)
    self.up4_conv =self.conv(1024,512)
    self.up_conv3 = self.up_conv(512, 256)
    self.up3_conv = self.conv(512,256)
    self.up_conv2 = self.up_conv(256,128)
    self.up2_conv = self.conv(256,128)
    self.up_conv1 = self.up_conv(128,64)
    self.up1_conv = self.conv(128,64)
    
    self.conv_1x1 = nn.Conv2d(64,out_channel,kernel_size=1)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    out1 = self.conv1(x)
    
    out2 = self.conv1_maxpool(out1)
    out2 = self.conv2(out2)
    
    out3 = self.conv2_maxpool(out2)
    out3 = self.conv3(out3)
    
    out4 = self.conv3_maxpool(out3)
    out4 = self.conv4(out4)
    
    out5 = self.conv4_maxpool(out4)
    out5 = self.conv5(out5)
    
    exp5 = self.up_conv4(out5)
    exp5 = torch.cat((out4, exp5), dim=1)
    exp5 = self.up4_conv(exp5)
    
    exp4 = self.up_conv3(exp5)
    exp4 = torch.cat((out3, exp4), dim=1)
    exp4 = self.up3_conv(exp4)
    
    exp3 = self.up_conv2(exp4)
    exp3 = torch.cat((out2, exp3), dim=1)
    exp3 = self.up2_conv(exp3)
    
    exp2 = self.up_conv1(exp3)
    exp2 = torch.cat((out1, exp2), dim=1)
    exp2 = self.up1_conv(exp2)
    
    exp1 = self.conv_1x1(exp2)
    exp1 = self.sigmoid(exp1)
    return exp1
    
    


In [ ]:
class MyDataset(Dataset):

    def __init__(self, len, home_directory, noise=2, mode="Train"):
      self.len = len
      self.examples = []
      self.iter_index = 0
      self.X = torch.empty((len, 128,128))
      self.Y = torch.empty((len,128,128), dtype=torch.long)
      self.input_directory = os.path.join(home_directory, mode, 'input')
      self.mask_directory = os.path.join(home_directory, mode, 'mask')
      
      print("dataset input path {}".format(self.input_directory))
      print("dataset mask path {}".format(self.mask_directory))
      
      input_names = os.listdir(self.input_directory)
      input_names.sort()
      mask_names = os.listdir(self.mask_directory)
      mask_names.sort()
      
      self.set_dataset(self.input_directory, input_names, True)
      self.set_dataset(self.mask_directory, mask_names, False)

      
    def set_dataset(self, directory, names, input_na = True):
      # print(self.len)
      # print(len(names))
      # print(names)
      index = 0
      for name in names:
        img_path = directory + '/' + name
        img = cv2.imread(img_path) 
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img =img/255
        resize_img = cv2.resize(img, (128,128))
        if input_na:
          # print(index)
          self.X[index] = torch.tensor(resize_img)
        else:
          
          resize_img = torch.from_numpy(resize_img).float()
          self.Y[index] = resize_img
        index  += 1  
      
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return (self.X[idx], self.Y[idx])
      


In [ ]:
dataset_train = MyDataset(60,'/content/drive/My Drive/A3/cat_data/cat_data')
trainloader = torch.utils.data.DataLoader(dataset_train, batch_size=20, shuffle=True)

# dataset_test = MyDataset(20,'/content/drive/My Drive/A3/cat_data/cat_data', 'Test')

model = UnetModel(1, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.7)

epochs = 10
model.train()
a = True
for e in range(epochs):
  running_loss = 0
  for images, labels in tqdm(trainloader):
    optimizer.zero_grad()
    images = images.unsqueeze(1)
    labels = labels.unsqueeze(1)
    labels = labels.float()
    log_ps = model(images)
    loss = criterion(log_ps, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
  else:
    print(f"Traning loss: {running_loss/len(trainloader)}")


dataset input path /content/drive/My Drive/A3/cat_data/cat_data/Train/input
dataset mask path /content/drive/My Drive/A3/cat_data/cat_data/Train/mask


100%|██████████| 3/3 [01:33<00:00, 31.17s/it]


Traning loss: 0.4460305670897166


100%|██████████| 3/3 [01:31<00:00, 30.36s/it]


Traning loss: 0.613702396551768


...

Traning loss: 0.6137023766835531


100%|██████████| 3/3 [01:29<00:00, 29.98s/it]


Traning loss: 0.613702396551768


100%|██████████| 3/3 [01:29<00:00, 30.00s/it]

Traning loss: 0.613702396551768


In [ ]:
dataset_test = MyDataset(21,'/content/drive/My Drive/A3/cat_data/cat_data', mode='Test')
testloader = torch.utils.data.DataLoader(dataset_test, batch_size=20, shuffle=True)
with torch.no_grad():
  for images, labels in tqdm(testloader):
      optimizer.zero_grad()
      images = images.unsqueeze(1)
      labels = labels.unsqueeze(1)
      log_ps = model(images)
      loss = criterion(log_ps, labels)
      running_loss += loss.item()
print(f"Test loss: {running_loss/len(testloader)}")

dataset input path /content/drive/My Drive/A3/cat_data/cat_data/Test/input
dataset mask path /content/drive/My Drive/A3/cat_data/cat_data/Test/mask


100%|██████████| 2/2 [00:10<00:00,  5.38s/it]

Test loss: 2.2097961604595184
